***GENERATED CODE FOR earthquakeregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import os
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs_server = str(os.environ['HDFS_SERVER'])
        hdfs_port = int(os.environ['HDFS_PORT'])
        df = spark.read.options(header='true', inferschema='true').csv(
            f"hdfs://{hdfs_server}:{hdfs_port}{eval(config)['url']}", header='true')
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
from pyspark.ml.feature import StringIndexer
import json
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def cleanValueForFE(self, value):
        if value == None:
            return ""
        elif str(value) == 'nan':
            return "nan"
        else:
            return value

    def replaceByMean(self, feature, df, mean_=-1):
        df1 = df
        df1 = df1.dropna()
        meanValue = self.cleanValueForFE(df1.select(
            mean(col(feature.name)).alias('mean')).collect()[0]["mean"])
        df = df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                      meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        df1 = df
        df1 = df1.dropna()
        maxValue = self.cleanValueForFE(df1.select(
            max(col(feature.name)).alias('max')).collect()[0]["max"])
        df = df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        df1 = df
        df1 = df1.dropna()
        minValue = self.cleanValueForFE(df1.select(
            min(col(feature.name)).alias('min')).collect()[0]["min"])
        df = df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        df1 = df
        df1 = df1.dropna()
        stddevValue = self.cleanValueForFE(df1.select(
            stddev(col(feature.name)).alias('stddev')).collect()[0]["stddev"])
        df = df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        df1 = df
        df1 = df1.dropna()
        fillValue = self.cleanValueForFE(
            df.where(col(feature.name).isNotNull()).head(1)[0][feature.name])
        df = df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Time', 'transformation_label': 'Extract Date'}], 'feature': 'Time', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Time'}, {'feature_label': 'Time', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Time')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Place', 'transformation_label': 'String Indexer'}], 'feature': 'Place', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '474', 'mean': '', 'stddev': '', 'min': '1 km S of Huangala, Peru', 'max': 'west of Macquarie Island', 'missing': '1', 'distinct': '358'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Place'}, {'feature_label': 'Place', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Place')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Latitude, Longitude', 'transformation_label': 'String Indexer'}], 'feature': 'Latitude, Longitude', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '-0.045, 125.023', 'max': '9.796, -43.992', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Latitude, Longitude'}, {'feature_label': 'Latitude, Longitude', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Latitude, Longitude')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'MagType', 'transformation_label': 'String Indexer'}], 'feature': 'MagType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Mi', 'max': 'mww', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'MagType'}, {'feature_label': 'MagType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('MagType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'net', 'transformation_label': 'String Indexer'}], 'feature': 'net', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'nc', 'max': 'us', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'net'}, {'feature_label': 'net', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('net')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ID', 'transformation_label': 'String Indexer'}], 'feature': 'ID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'nc73821036', 'max': 'usd000h551', 'missing': '0', 'distinct': '500'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ID'}, {'feature_label': 'ID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ID')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Updated', 'transformation_label': 'Extract Date'}], 'feature': 'Updated', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Updated'}, {'feature_label': 'Updated', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Updated')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': '_c15', 'transformation_label': 'String Indexer'}], 'feature': '_c15', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '0', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '500', 'distinct': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': '_c15'}, {'feature_label': '_c15', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('_c15')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Type', 'transformation_label': 'String Indexer'}], 'feature': 'Type', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'earthquake', 'max': 'nuclear explosion', 'missing': '0', 'distinct': '3'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Type'}, {'feature_label': 'Type', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Type')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'status', 'transformation_label': 'String Indexer'}], 'feature': 'status', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'reviewed', 'max': 'reviewed', 'missing': '0', 'distinct': '1'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'status'}, {'feature_label': 'status', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('status')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'locationSource', 'transformation_label': 'String Indexer'}], 'feature': 'locationSource', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'nc', 'max': 'us', 'missing': '0', 'distinct': '9'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'locationSource'}, {'feature_label': 'locationSource', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('locationSource')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'magSource', 'transformation_label': 'String Indexer'}], 'feature': 'magSource', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'nc', 'max': 'us', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'magSource'}, {'feature_label': 'magSource', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('magSource')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15, use_dask=True)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[1]').getOrCreate()
#%run earthquakeregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	significantearthquakedataset = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/Significant Earthquake Dataset 1900-2023 - Significant Earthquake Dataset 1900-2023.csv', 'filename': 'Significant Earthquake Dataset 1900-2023 - Significant Earthquake Dataset 1900-2023.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'domain': 'http://172.31.59.158', 'port': '40070', 'dirPath': '/FileStore/platform', 'server_url': '/nexusMax/NexusMaxPlatform/uploads/platform/'}")
	#sourcePostExecutionHook(significantearthquakedataset)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***TRANSFORMING DATAFRAME***

In [ ]:
#%run earthquakeregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	earthquakeregressionautofe = TransformationMain.run(significantearthquakedataset,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "Time", "transformation_label": "Extract Date"}], "feature": "Time", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Time"}, {"transformationsData": [{"feature_label": "Place", "transformation_label": "String Indexer"}], "feature": "Place", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "474", "mean": "", "stddev": "", "min": "1 km S of Huangala, Peru", "max": "west of Macquarie Island", "missing": "1", "distinct": "358"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Place"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Latitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.8", "stddev": "30.16", "min": "-60.748", "max": "85.047", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Latitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Longitude", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "29.59", "stddev": "120.96", "min": "-179.9677", "max": "179.9981", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Longitude"}, {"transformationsData": [{"feature_label": "Latitude, Longitude", "transformation_label": "String Indexer"}], "feature": "Latitude, Longitude", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "-0.045, 125.023", "max": "9.796, -43.992", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Latitude, Longitude"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Depth", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "61.82", "stddev": "122.47", "min": "0.0", "max": "660.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Depth"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Mag", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5.86", "stddev": "0.43", "min": "5.5", "max": "7.9", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Mag"}, {"transformationsData": [{"feature_label": "MagType", "transformation_label": "String Indexer"}], "feature": "MagType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Mi", "max": "mww", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "MagType"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "nst", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "356", "mean": "134.92", "stddev": "65.11", "min": "21", "max": "452", "missing": "144"}, "updatedLabel": "nst"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "gap", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "46.34", "stddev": "28.79", "min": "10.0", "max": "214.0", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "gap"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "dmin", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "4.63", "stddev": "6.09", "min": "0.058", "max": "32.846", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "dmin"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "rms", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "0.75", "stddev": "0.23", "min": "0.18", "max": "2.82", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "rms"}, {"transformationsData": [{"feature_label": "net", "transformation_label": "String Indexer"}], "feature": "net", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "nc", "max": "us", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "net"}, {"transformationsData": [{"feature_label": "ID", "transformation_label": "String Indexer"}], "feature": "ID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "nc73821036", "max": "usd000h551", "missing": "0", "distinct": "500"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ID"}, {"transformationsData": [{"feature_label": "Updated", "transformation_label": "Extract Date"}], "feature": "Updated", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Updated"}, {"transformationsData": [{"feature_label": "_c15", "transformation_label": "String Indexer"}], "feature": "_c15", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "0", "mean": "", "stddev": "", "min": "", "max": "", "missing": "500", "distinct": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "_c15"}, {"transformationsData": [{"feature_label": "Type", "transformation_label": "String Indexer"}], "feature": "Type", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "earthquake", "max": "nuclear explosion", "missing": "0", "distinct": "3"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Type"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "horizontalError", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "7.5", "stddev": "2.25", "min": "0.67", "max": "15.39", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "horizontalError"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "depthError", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "499", "mean": "2.3", "stddev": "1.51", "min": "0.4", "max": "26.8", "missing": "1"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "depthError"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "magError", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "498", "mean": "0.06", "stddev": "0.02", "min": "0.021", "max": "0.12", "missing": "2"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "magError"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "magNst", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "44.96", "stddev": "61.95", "min": "0", "max": "761", "missing": "0"}, "updatedLabel": "magNst"}, {"transformationsData": [{"feature_label": "status", "transformation_label": "String Indexer"}], "feature": "status", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "reviewed", "max": "reviewed", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "status"}, {"transformationsData": [{"feature_label": "locationSource", "transformation_label": "String Indexer"}], "feature": "locationSource", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "nc", "max": "us", "missing": "0", "distinct": "9"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "locationSource"}, {"transformationsData": [{"feature_label": "magSource", "transformation_label": "String Indexer"}], "feature": "magSource", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "nc", "max": "us", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "magSource"}]}))

	#transformationPostExecutionHook(earthquakeregressionautofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run earthquakeregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(earthquakeregressionautofe, ["Latitude", "Longitude", "Depth", "Mag", "nst", "gap", "dmin", "rms", "horizontalError", "depthError", "magError", "magNst", "Time_dayofmonth", "Time_month", "Time_year", "Place_stringindexer", "MagType_stringindexer", "net_stringindexer", "ID_stringindexer", "Updated_dayofmonth", "Updated_month", "Updated_year", "_c15_stringindexer", "Type_stringindexer", "status_stringindexer", "locationSource_stringindexer", "magSource_stringindexer"], "Latitude, Longitude_stringindexer")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
#spark.stop()


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

spark.stop()

